In [1]:
import pandas as pd
import os

In [2]:
counts_folder = './data/counts/TARA/'

In [3]:
# Load metadata dataframes and renames columns

# TARA metadata
metadata_file = "./data/TARA_metadata.csv"
metadata_df = pd.read_csv(metadata_file)
metadata_df = metadata_df.rename(columns = {'ENA-RUN':'dataset', "ENA-SPOT-COUNT":"dataset_size"})
metadata_df_slim = metadata_df[["dataset", "Station", "Longitude Start", "Latitude Start", "Depth", "Protocol Label", "dataset_size"]]

# Info of the genomes used for mapping
genome_info_tsv = './data/genomes_names.20170109.mgii_virus_size_corrected.tsv'
genome_info_df = pd.read_csv(genome_info_tsv, sep = '\t', index_col=0)

In [4]:
# Concatenates the different tsv files with the counts into a tidy dataframe
counts_df = pd.DataFrame()
for path, dirs, files in os.walk(counts_folder):
    for filename in files:
        dataset_name = filename.split('.')[0]
        file_path = os.path.join(path, filename)
        dataset_df = pd.read_csv(file_path, sep='\t', names=['genome_hash','count'], dtype={'genome_hash':str,'count':pd.np.int})
        dataset_df['dataset'] = dataset_name
        
        counts_df = pd.concat([counts_df, dataset_df])

In [5]:
# Add metadata columns
counts_df = counts_df.merge(metadata_df_slim, on='dataset', how='left')
counts_df = counts_df.merge(genome_info_df, on='genome_hash', how='left')

In [6]:
# Count normalization
counts_df['n_count'] = counts_df['count']/((counts_df['genome_size']/1000) * (counts_df['dataset_size']/2/1000000))

In [7]:
# Tags viral datasets
viral_criterion_fraction = counts_df['Protocol Label'].map(lambda x: x in {'GIRUS_NUC-dry_W0.1-0.22','VIRUS_NUC-DNA-Fe(20L)_W<-0.22','VIRUS_NUC-Fe_W<-0.22'})

In [8]:
# Name the Fraction of origin
counts_df.loc[viral_criterion_fraction,'Fraction'] = 'VIRUS'
counts_df.loc[viral_criterion_fraction == False,'Fraction'] = 'BACT'

In [9]:
# List to reorder dataframe
column_order = [u'genome_group',u'genome_hash', u'count', u'n_count', u'Fraction', u'dataset', u'Station', u'Latitude Start',
       u'Longitude Start', u'Depth', u'Protocol Label', u'dataset_size', u'genome_name',u'genome_size']

In [10]:
counts_df = counts_df[column_order]

In [11]:
counts_df.describe()

,count,n_count,Latitude Start,Longitude Start,Depth,dataset_size,genome_size
count,1.025430e+05,102543.000000,102543.000000,102543.000000,102543.000000,1.025430e+05,1.025430e+05
mean,3.004476e+04,0.884748,-3.038715,-31.783807,143.609023,9.305616e+07,3.200314e+05
std,4.267505e+05,6.843985,25.556760,73.070630,243.561214,6.574257e+07,6.639606e+05
min,0.000000e+00,0.000000,-62.223100,-159.046000,5.000000,2.527066e+06,5.030000e+02
25%,0.000000e+00,0.000000,-21.029200,-96.023200,5.000000,3.804226e+07,2.561100e+04
50%,3.600000e+01,0.016568,-8.910900,-17.909900,30.000000,5.255640e+07,7.028500e+04
75%,6.950000e+02,0.181444,18.734100,32.898000,120.000000,1.646200e+08,1.614400e+05
max,3.779579e+07,617.416016,43.768400,73.906700,1000.000000,2.695569e+08,3.148033e+06


In [12]:
counts_df.to_csv('normalized_reads.tara.20170115.tsv', sep='\t')